# Prep CONUS use-case.

This uses the GeoSpatialFabric v1.1.

In [1]:
import geopandas as gpd
import requests
import pandas as pd
import xarray as xr

In [2]:
root_dir = '/mnt/d/oNHM/NHM_PRMS_CONUS_GF_1_1/'

In [3]:
gdf = gpd.read_file(root_dir + "GIS/GFv1.1_nhru_simp.shp")
gdf

,OBJECTID,nhru_v1_1,hru_segmen,nhm_id,hru_id_nat,Version,Shape_Leng,Shape_Area,Change,geometry
0,1.0,76127,40038,76128,76128,1,80441.423292,1.881188e+08,-0.017302,"POLYGON ((-105544.567 804074.976, -105415.315 ..."
1,2.0,76147,40038,76148,76148,1,53413.506290,4.418597e+07,0.054540,"POLYGON ((-97185.217 806355.005, -97154.773 80..."
2,3.0,76170,40021,76171,76171,1,54988.828358,7.338919e+07,0.018316,"MULTIPOLYGON (((-105894.643 815045.861, -10570..."
3,4.0,76172,40019,76173,76173,1,31915.213081,3.629082e+07,0.049861,"POLYGON ((-106762.641 815925.020, -106754.831 ..."
4,5.0,76181,40019,76182,76182,1,27390.023934,1.567346e+07,0.154732,"POLYGON ((-109785.311 816675.047, -109784.942 ..."
...,...,...,...,...,...,...,...,...,...,...
114953,114954.0,57964,31028,57965,57965,1,117117.944712,2.550804e+08,0.002392,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,114955.0,64080,28886,64081,64081,1,67362.397583,1.102741e+08,0.002877,"POLYGON ((-1284135.000 2658485.000, -1284035.0..."
114955,114956.0,64150,28866,64151,64151,1,80355.338777,1.884838e+08,0.022769,"POLYGON ((-1347645.097 2651834.819, -1347614.8..."
114956,114957.0,65633,31412,65634,65634,1,71022.681331,7.386700e+07,-0.005068,"POLYGON ((-1017265.000 2869285.000, -1017185.0..."


In [4]:
gdf.columns

Index(['OBJECTID', 'nhru_v1_1', 'hru_segmen', 'nhm_id', 'hru_id_nat',
       'Version', 'Shape_Leng', 'Shape_Area', 'Change', 'geometry'],
      dtype='object')

Simplify file for size

In [5]:
ngdf = gdf[["nhru_v1_1", "geometry"]]
ngdf


,nhru_v1_1,geometry
0,76127,"POLYGON ((-105544.567 804074.976, -105415.315 ..."
1,76147,"POLYGON ((-97185.217 806355.005, -97154.773 80..."
2,76170,"MULTIPOLYGON (((-105894.643 815045.861, -10570..."
3,76172,"POLYGON ((-106762.641 815925.020, -106754.831 ..."
4,76181,"POLYGON ((-109785.311 816675.047, -109784.942 ..."
...,...,...
114953,57964,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,64080,"POLYGON ((-1284135.000 2658485.000, -1284035.0..."
114955,64150,"POLYGON ((-1347645.097 2651834.819, -1347614.8..."
114956,65633,"POLYGON ((-1017265.000 2869285.000, -1017185.0..."


In [6]:
# Find invalid geometries
invalid_geometries = ngdf[~ngdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

# Correct invalid geometries by explicitly targeting the original DataFrame
ngdf.loc[~ngdf.is_valid, 'geometry'] = ngdf.loc[~ngdf.is_valid, 'geometry'].apply(lambda geom: geom.buffer(0))

# Re-check for invalid geometries after the correction
invalid_geometries_after = ngdf[~ngdf.is_valid]
print(f"Found {len(invalid_geometries_after)} invalid geometries after correction.")


Found 9988 invalid geometries.
Found 0 invalid geometries after correction.


In [7]:
ngdf.crs

<Projected CRS: ESRI:102039>
Name: USA_Contiguous_Albers_Equal_Area_Conic_USGS_version
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
ggdf = ngdf.to_crs("EPSG:4326")
ggdf

,nhru_v1_1,geometry
0,76127,"POLYGON ((-97.09918 30.30709, -97.09779 30.303..."
1,76147,"POLYGON ((-97.01238 30.32845, -97.01206 30.328..."
2,76170,"MULTIPOLYGON (((-97.10415 30.40583, -97.10214 ..."
3,76172,"POLYGON ((-97.11331 30.41365, -97.11322 30.413..."
4,76181,"POLYGON ((-97.14492 30.42008, -97.14461 30.420..."
...,...,...
114953,57964,"MULTIPOLYGON (((-99.89073 47.49114, -99.89112 ..."
114954,64080,"POLYGON ((-112.61378 45.90173, -112.61236 45.9..."
114955,64150,"POLYGON ((-113.40203 45.74099, -113.40395 45.7..."
114956,65633,"POLYGON ((-109.60026 48.16516, -109.59892 48.1..."


## Step 2: Sort the DataFrame by 'nhm_id'
Looks like data are sorted already but we'll sort anyway. CBH data supplied to this model should be sorted by nhm_id.

In [10]:
gdf_sorted = ggdf.sort_values(by="nhru_v1_1")
gdf_sorted

,nhru_v1_1,geometry
101643,1,"POLYGON ((-73.38688 41.09690, -73.38721 41.094..."
101644,2,"POLYGON ((-73.38531 41.13467, -73.38783 41.133..."
101645,3,"POLYGON ((-73.41946 41.16067, -73.41981 41.159..."
101646,4,"POLYGON ((-73.41096 41.15832, -73.41061 41.157..."
101647,5,"MULTIPOLYGON (((-73.28636 41.12780, -73.28627 ..."
...,...,...
84543,114954,"POLYGON ((-122.65566 49.13672, -122.65573 49.1..."
109623,114955,"POLYGON ((-68.70503 44.50792, -68.70503 44.506..."
109622,114956,"POLYGON ((-69.76945 44.23066, -69.76737 44.228..."
85863,114957,"MULTIPOLYGON (((-103.79920 48.63538, -103.7989..."


## Step 3: Export "nhm_id" to a file

In [12]:
gdf_sorted['nhru_v1_1'].to_csv(root_dir + 'daily/input/nhm_id', index=False, header=False)

## Step 4: Calculate and write hru centroids.

In [13]:
centroids = gdf_sorted.to_crs(5070).geometry.centroid
centroids_ll = centroids.to_crs(4326)
lats = centroids_ll.y
lons = centroids_ll.x
lats.to_csv(root_dir + 'daily/input/hru_lat.txt', index=False, header=False)
lons.to_csv(root_dir + 'daily/input/hru_lon.txt', index=False, header=False)

In [14]:
points = list(zip(lons, lats))
points[-10:]

[(-122.61315237403804, 48.44904319371622),
 (-122.58755449074951, 48.47586908442381),
 (-122.62319168667607, 48.550440241670245),
 (-122.7172222640981, 49.05756732423108),
 (-122.70882550085287, 49.02220151090653),
 (-122.8658175756982, 49.11218765110783),
 (-68.79187020242308, 44.49833446463939),
 (-69.79335554545511, 44.1279237043866),
 (-103.58149794429606, 48.65201330460139),
 (-119.23161441119703, 46.17897523854811)]

## 5 Get elevations - used for calculating humidity from cfsv2 data.

Elevation is required for humidity conversion so we take the elevation of the hru centroid.

In [15]:
import rasterio
from rasterio.transform import rowcol
import numpy as np

# List of GeoTIFF files
tiff_files = ['/mnt/d/oNHM/gt30w100n40.tif', '/mnt/d/oNHM/gt30w100n90.tif', '/mnt/d/oNHM/gt30w140n40.tif', '/mnt/d/oNHM/gt30w140n90.tif']


# Function to find elevation for a single point
def get_elevation(lon, lat, datasets):
    for dataset in datasets:
        # Check if the point is within the bounds of this dataset
        if dataset.bounds[0] <= lon <= dataset.bounds[2] and dataset.bounds[1] <= lat <= dataset.bounds[3]:
            # Convert the lon/lat to row/col in the dataset
            row, col = rowcol(dataset.transform, lon, lat)
            # Read the elevation value at this row/col
            value = dataset.read(1)[row, col]
            return value
    return np.nan  # Return NaN if the point is not in any dataset

# Open all the TIFFs
datasets = [rasterio.open(tiff) for tiff in tiff_files]

# Get elevations for all points
elevations = [get_elevation(lon, lat, datasets) for lon, lat in points]

# Don't forget to close the datasets
for dataset in datasets:
    dataset.close()

print(elevations)


[16, 29, 39, 61, 11, 10, 66, 30, 15, 30, 13, 3, 22, 57, 1, 13, 25, 31, 34, 146, 60, 58, 164, 183, 202, 178, 169, 87, 152, 138, 213, 33, 30, 91, 152, 60, 30, 8, 79, 58, 89, 13, 21, 85, 156, 95, 61, 152, 60, 91, 60, 143, 20, 61, 124, 80, 103, 132, 89, 189, 82, 30, 90, 14, 75, 157, 31, 512, 131, 103, 132, 152, 8, 76, 81, 60, 172, 152, 135, 185, 204, 171, 426, 223, 92, 153, 46, 152, 454, 122, 93, 243, 487, 109, 117, 163, 152, 182, 10, 1, 402, 31, 138, 115, 145, 28, 31, 150, 426, 152, 136, 41, 243, 175, 404, 457, 142, 58, 152, 152, 148, 122, 422, 286, 46, 44, 152, 155, 213, 19, 18, 203, 372, 320, 80, 33, 396, 61, 54, 264, 60, 30, 90, 27, 28, 20, 30, 61, 404, 268, 152, 429, 278, 34, 91, 150, 182, 27, 84, 77, 60, 351, 479, 183, 182, 44, 396, 42, 43, 213, 471, 213, 61, 122, 143, 122, 69, 85, 122, 121, 372, 50, 396, 102, 249, 124, 60, 178, 365, 60, 111, 106, 82, 66, 214, 269, 330, 370, 145, 243, 182, 10, 394, 413, 257, 183, 208, 91, 81, 91, 1, 62, 94, 153, 173, 141, 61, 88, 128, 200, 35, 426, 6

In [16]:
df = pd.DataFrame(elevations)
df.to_csv(f'{root_dir}/daily/input/elevation', index=False, header=False)

## Repeat for segments

In [20]:
gdf = gpd.read_file('/mnt/d/oNHM/nhm_prms_conus_gf_1_1_n_segment.shp')
gdf

,nsegment_v,geometry
0,1,"LINESTRING Z (2101948.624 2876678.641 0.000, 2..."
1,2,"LINESTRING Z (2167789.031 2829021.852 0.000, 2..."
2,3,"LINESTRING Z (2131936.492 2865675.020 0.000, 2..."
3,4,"LINESTRING Z (2151719.943 2849594.051 0.000, 2..."
4,5,"LINESTRING Z (2155981.103 2842240.715 0.000, 2..."
...,...,...
58484,23754,"LINESTRING Z (129793.551 2795623.027 0.000, 12..."
58485,23491,"LINESTRING Z (-350068.189 2908499.924 0.000, -..."
58486,23401,"LINESTRING Z (-512215.528 3002734.106 0.000, -..."
58487,23434,"LINESTRING Z (-463677.806 2967339.542 0.000, -..."


In [27]:
gdf.set_crs("ESRI:102039", inplace=True)

,nsegment_v,geometry
0,1,"LINESTRING Z (2101948.624 2876678.641 0.000, 2..."
1,2,"LINESTRING Z (2167789.031 2829021.852 0.000, 2..."
2,3,"LINESTRING Z (2131936.492 2865675.020 0.000, 2..."
3,4,"LINESTRING Z (2151719.943 2849594.051 0.000, 2..."
4,5,"LINESTRING Z (2155981.103 2842240.715 0.000, 2..."
...,...,...
58484,23754,"LINESTRING Z (129793.551 2795623.027 0.000, 12..."
58485,23491,"LINESTRING Z (-350068.189 2908499.924 0.000, -..."
58486,23401,"LINESTRING Z (-512215.528 3002734.106 0.000, -..."
58487,23434,"LINESTRING Z (-463677.806 2967339.542 0.000, -..."


In [28]:
gdf.columns


Index(['nsegment_v', 'geometry'], dtype='object')

In [29]:
gdf_sorted = gdf.sort_values(by="nsegment_v")
gdf_sorted['nsegment_v'].to_csv(root_dir + 'daily/input/nhm_seg', index=False, header=False)
centroids = gdf_sorted.to_crs(5070).geometry.centroid
centroids_ll = centroids.to_crs(4326)
lats = centroids_ll.y
lons = centroids_ll.x
lats.to_csv(root_dir + 'daily/input/lat_seg.txt', index=False, header=False)
lons.to_csv(root_dir + 'daily/input/lon_seg.txt', index=False, header=False)

In [29]:
from gdptools import WeightGen
from gdptools import AggGen
from gdptools import ClimRCatData


In [50]:
gdf = gpd.read_file(root_dir + "GIS/GFv1.1_nhru_simp.shp")
gdf

,OBJECTID,nhru_v1_1,hru_segmen,nhm_id,hru_id_nat,Version,Shape_Leng,Shape_Area,Change,geometry
0,1.0,76127,40038,76128,76128,1,80441.423292,1.881188e+08,-0.017302,"POLYGON ((-105544.567 804074.976, -105415.315 ..."
1,2.0,76147,40038,76148,76148,1,53413.506290,4.418597e+07,0.054540,"POLYGON ((-97185.217 806355.005, -97154.773 80..."
2,3.0,76170,40021,76171,76171,1,54988.828358,7.338919e+07,0.018316,"MULTIPOLYGON (((-105894.643 815045.861, -10570..."
3,4.0,76172,40019,76173,76173,1,31915.213081,3.629082e+07,0.049861,"POLYGON ((-106762.641 815925.020, -106754.831 ..."
4,5.0,76181,40019,76182,76182,1,27390.023934,1.567346e+07,0.154732,"POLYGON ((-109785.311 816675.047, -109784.942 ..."
...,...,...,...,...,...,...,...,...,...,...
114953,114954.0,57964,31028,57965,57965,1,117117.944712,2.550804e+08,0.002392,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,114955.0,64080,28886,64081,64081,1,67362.397583,1.102741e+08,0.002877,"POLYGON ((-1284135.000 2658485.000, -1284035.0..."
114955,114956.0,64150,28866,64151,64151,1,80355.338777,1.884838e+08,0.022769,"POLYGON ((-1347645.097 2651834.819, -1347614.8..."
114956,114957.0,65633,31412,65634,65634,1,71022.681331,7.386700e+07,-0.005068,"POLYGON ((-1017265.000 2869285.000, -1017185.0..."


In [51]:
ngdf = gdf[["nhru_v1_1", "geometry"]]
ngdf

,nhru_v1_1,geometry
0,76127,"POLYGON ((-105544.567 804074.976, -105415.315 ..."
1,76147,"POLYGON ((-97185.217 806355.005, -97154.773 80..."
2,76170,"MULTIPOLYGON (((-105894.643 815045.861, -10570..."
3,76172,"POLYGON ((-106762.641 815925.020, -106754.831 ..."
4,76181,"POLYGON ((-109785.311 816675.047, -109784.942 ..."
...,...,...
114953,57964,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,64080,"POLYGON ((-1284135.000 2658485.000, -1284035.0..."
114955,64150,"POLYGON ((-1347645.097 2651834.819, -1347614.8..."
114956,65633,"POLYGON ((-1017265.000 2869285.000, -1017185.0..."


In [55]:
# Step 1: Dissolve by 'nhru_v1_1'
dissolved_gdf = ngdf.dissolve(by='nhru_v1_1')

# Step 2: Sort by 'nhru_v1_1' - Note: The dissolve operation sets 'nhru_v1_1' as the index.
dissolved_gdf = dissolved_gdf.sort_index()

# Step 3: Fix bad geometries
def fix_geometry(geom):
    if not geom.is_valid:
        return geom.buffer(0)
    return geom

dissolved_gdf['geometry'] = dissolved_gdf['geometry'].apply(fix_geometry)

# If you want to reset the index to make 'nhru_v1_1' a column again
dissolved_gdf.reset_index(inplace=True)
dissolved_gdf

,nhru_v1_1,geometry
0,1,"POLYGON ((1866614.821 2232974.892, 1866644.832..."
1,2,"POLYGON ((1865744.747 2237114.794, 1865565.329..."
2,3,"POLYGON ((1862295.252 2239274.962, 1862294.915..."
3,4,"POLYGON ((1863045.106 2239185.024, 1863104.894..."
4,5,"MULTIPOLYGON (((1873934.921 2238284.871, 18739..."
...,...,...
114953,114954,"POLYGON ((-1944885.000 3177795.000, -1944905.0..."
114954,114955,"POLYGON ((2135385.000 2699714.735, 2135415.000..."
114955,114956,"POLYGON ((2062701.753 2646525.047, 2062939.750..."
114956,114957,"MULTIPOLYGON (((-551985.283 2866304.985, -5521..."


In [54]:
dissolved_gdf

In [57]:
dissolved_gdf = dissolved_gdf.rename(columns={"nhru_v1_1": "nhm_id"})
dissolved_gdf

,nhm_id,geometry
0,1,"POLYGON ((1866614.821 2232974.892, 1866644.832..."
1,2,"POLYGON ((1865744.747 2237114.794, 1865565.329..."
2,3,"POLYGON ((1862295.252 2239274.962, 1862294.915..."
3,4,"POLYGON ((1863045.106 2239185.024, 1863104.894..."
4,5,"MULTIPOLYGON (((1873934.921 2238284.871, 18739..."
...,...,...
114953,114954,"POLYGON ((-1944885.000 3177795.000, -1944905.0..."
114954,114955,"POLYGON ((2135385.000 2699714.735, 2135415.000..."
114955,114956,"POLYGON ((2062701.753 2646525.047, 2062939.750..."
114956,114957,"MULTIPOLYGON (((-551985.283 2866304.985, -5521..."


In [58]:
dissolved_gdf.to_feather(root_dir + 'GIS/model_nhru.feather')

In [59]:
gdf = gpd.read_feather(root_dir + 'GIS/model_nhru.feather')
gdf

,nhm_id,geometry
0,1,"POLYGON ((1866614.821 2232974.892, 1866644.832..."
1,2,"POLYGON ((1865744.747 2237114.794, 1865565.329..."
2,3,"POLYGON ((1862295.252 2239274.962, 1862294.915..."
3,4,"POLYGON ((1863045.106 2239185.024, 1863104.894..."
4,5,"MULTIPOLYGON (((1873934.921 2238284.871, 18739..."
...,...,...
114953,114954,"POLYGON ((-1944885.000 3177795.000, -1944905.0..."
114954,114955,"POLYGON ((2135385.000 2699714.735, 2135415.000..."
114955,114956,"POLYGON ((2062701.753 2646525.047, 2062939.750..."
114956,114957,"MULTIPOLYGON (((-551985.283 2866304.985, -5521..."


In [60]:
climater_cat = "https://github.com/mikejohnson51/climateR-catalogs/releases/download/March-2024/catalog.parquet"
cat = pd.read_parquet(climater_cat)
cat.head()

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
0,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp45_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
1,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp85_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
2,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp26_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
3,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp45_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
4,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp60_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX


In [61]:
_id = "gridmet"
_varname = "tmmx"

# an example query returns a pandas dataframe.
tc = cat.query(
    "id == @_id & variable == @_varname"
)
tc

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
13997,gridmet,None,http://thredds.northwestknowledge.net:8080/thr...,opendap,daily_maximum_temperature,tmmx,tmmx,K,None,None,...,49.4,25.066667,0.041667,0.041667,1386.0,585.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,False,,TYX


In [62]:
# Create a dictionary of parameter dataframes for each variable
tvars = ["tmmx"]
cat_params = [cat.query("id == @_id & variable == @_var").to_dict(orient="records")[0] for _var in tvars]

cat_dict = dict(zip(tvars, cat_params))

# Output an example of the cat_param.json entry for "aet".
cat_dict.get("tmmx")

{'id': 'gridmet',
 'asset': None,
 'URL': 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_tmmx_1979_CurrentYear_CONUS.nc',
 'type': 'opendap',
 'varname': 'daily_maximum_temperature',
 'variable': 'tmmx',
 'description': 'tmmx',
 'units': 'K',
 'model': None,
 'ensemble': None,
 'scenario': None,
 'T_name': 'day',
 'duration': '1979-01-01/..',
 'interval': '1 days',
 'nT': nan,
 'X_name': 'lon',
 'Y_name': 'lat',
 'X1': -124.76666663333334,
 'Xn': -67.05833330000002,
 'Y1': 49.400000000000006,
 'Yn': 25.066666666666666,
 'resX': 0.041666666666666664,
 'resY': 0.04166666666666668,
 'ncols': 1386.0,
 'nrows': 585.0,
 'crs': '+proj=longlat +a=6378137 +f=0.00335281066474748 +pm=0 +no_defs',
 'toptobottom': False,
 'tiled': '',
 'dim_order': 'TYX'}

In [63]:
from gdptools import WeightGen
from gdptools import AggGen
from gdptools import ClimRCatData


In [65]:
user_data = ClimRCatData(
    cat_dict=cat_dict,
    f_feature=gdf,
    id_feature='nhm_id',
    period=["1980-01-01", "1980-12-31"]
)

wght_gen = WeightGen(
    user_data=user_data,
    method="parallel",
    output_file=root_dir + "GIS/conus_weights.csv",
    weight_gen_crs=6931,
    jobs=4
)

wghts = wght_gen.calculate_weights(intersections=False)

Using parallel engine
grid cells generated in 98.5787 seconds
Data preparation finished in 99.1911 seconds
Reprojecting to epsg:EPSG:6931 finished in 4.86 second(s)
Validating polygons
     - validating source polygons
     - fixing 0 invalid polygons.
     - validating target polygons
     - fixing 3 invalid polygons.
Validate polygons finished in 5.8354 seconds
Weight gen finished in 71.2427 seconds
